In [1]:
import pandas as pd
# from feature_cross import *
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

/home/sf/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
d = pd.Series([i for i in range(100)])
d.unique().size
df=pd.DataFrame()
df['value']=  d
df['label']=df.value.apply(lambda x: 1 if x>34 else 0)
df['rank_col'] = df['value'].rank(method='max') 
df
# df['rank_col_1'] = df['rank_col'].apply(lambda x: int(x) if x > 0 else -1)
# df
# d = d.apply(lambda x: 1 if x>3 else 0)
# t= np.mat(d).T
# np.vstack((np.mat(['label']),t)).shape
# d.values
# m=np.mat(df)
# m[:,[2,3]]

,value,label,rank_col
0,0,0,1.0
1,1,0,2.0
2,2,0,3.0
3,3,0,4.0
4,4,0,5.0
5,5,0,6.0
6,6,0,7.0
7,7,0,8.0
8,8,0,9.0
9,9,0,10.0


In [2]:
print "======================正在读取文件，请稍后......=============================="
path1 = "/home/sf/work/data/zhima_score_weibo_text_describe.csv"#微博特征
feat=pd.read_csv(path1,index_col=0)
feat = feat.dropna(subset=['user_id','sum_reposts'])

label = feat['score']
X = feat.iloc[:,1:]

======================正在读取文件，请稍后......==============================


In [3]:
trian_X,val_X,trian_Y,val_Y=train_test_split(X,label,test_size=0.3)

dval = xgb.DMatrix(val_X,label=val_Y)
dtrian = xgb.DMatrix(trian_X,label=trian_Y) 

params = {
        'booster': 'gbtree',
        'objective': 'reg:linear',
        # 'scale_pos_weight': float(len(y)-sum(y))/float(sum(y)),
        'eval_metric': 'mae',
        'gamma': 0.01,
        'silent': 1,
        'max_depth': 5,
        'lambda': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.2,
        'min_child_weight': 0.1,
        'eta': 0.04,
        'seed': 3,
        'nthread': 5
    }

watchlist = [(dtrian, 'train'), (dval, 'val')]
model = xgb.train(params, dtrian, num_boost_round=1000,evals=watchlist, verbose_eval=200,early_stopping_rounds=100)

[0]	train-mae:671.011	val-mae:670.606
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 100 rounds.
[200]	train-mae:40.7954	val-mae:41.2357
[400]	train-mae:40.3724	val-mae:41.0946
Stopping. Best iteration:
[429]	train-mae:40.3319	val-mae:41.0904



In [8]:
# s = pd.DataFrame.from_dict(model.get_score(importance_type="gain"), orient='index')
# print model.get_score(importance_type="gain")
# print model.get_score()
help(model)

Help on Booster in module xgboost.core object:

class Booster(__builtin__.object)
 |  "A Booster of of XGBoost.
 |  
 |  Booster is the model of xgboost, that contains low level routines for
 |  training, prediction and evaluation.
 |  
 |  Methods defined here:
 |  
 |  __copy__(self)
 |  
 |  __deepcopy__(self, _)
 |  
 |  __del__(self)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, params=None, cache=(), model_file=None)
 |      Initialize the Booster.
 |      
 |      Parameters
 |      ----------
 |      params : dict
 |          Parameters for boosters.
 |      cache : list
 |          List of cache items.
 |      model_file : string
 |          Path to the model file.
 |  
 |  __setstate__(self, state)
 |  
 |  attr(self, key)
 |      Get attribute string from the Booster.
 |      
 |      Parameters
 |      ----------
 |      key : str
 |          The key to get attribute from.
 |      
 |      Returns
 |      -------
 |      value : str
 |          The attribute value of 

In [11]:
model.eval_set()

TypeError: eval_set() takes at least 2 arguments (1 given)

In [55]:
X_M = np.mat(X)
print X_M.shape
np.array(X_M[0,:])
pd.DataFrame(X_M[1:,:],columns=np.array(X_M[0,:])[0])

(145066, 16)


,1000063771.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333333333,0.0,0.0,0.0,0.0416666666667,0.0,24.0
0,1.000091e+09,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,2.0
1,1.000165e+09,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.0,100.0
2,1.000246e+09,0.0,4.0,4.0,0.0,6.0,0.0,8.0,0.000000,0.333333,0.333333,0.0,0.000000,0.666667,0.0,12.0
3,1.000315e+09,1.0,2.0,9.0,0.0,85.0,0.0,96.0,0.010204,0.020408,0.091837,0.0,0.000000,0.979592,0.0,98.0
4,1.000324e+09,2.0,1.0,1.0,0.0,8.0,0.0,17.0,0.100000,0.050000,0.050000,0.0,0.000000,0.850000,0.0,20.0
5,1.000434e+09,2.0,2.0,19.0,0.0,73.0,0.0,88.0,0.020000,0.020000,0.190000,0.0,0.000000,0.880000,0.0,100.0
6,1.000455e+09,8.0,6.0,0.0,0.0,3.0,0.0,3.0,2.000000,1.500000,0.000000,0.0,0.000000,0.750000,0.0,4.0
7,1.000466e+09,0.0,3.0,0.0,0.0,50.0,0.0,82.0,0.000000,0.030303,0.000000,0.0,0.000000,0.828283,0.0,99.0
8,1.000534e+09,0.0,3.0,0.0,0.0,2.0,0.0,3.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.0,3.0
9,1.000540e+09,0.0,0.0,0.0,0.0,3.0,0.0,7.0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,7.0


In [15]:
s = pd.DataFrame.from_dict(model.get_score(importance_type="gain"), orient='index')
s.columns = ['weight']
s2 = pd.DataFrame.from_dict(model.get_score(importance_type="gain"), orient='index')
s2.columns = ['weight']
f = s2+s
np.mat(f)

matrix([[  14683.22454178],
        [  44011.85841024],
        [  45430.65029829],
        [ 201868.65811849],
        [  78486.59681224],
        [  69842.19107493],
        [  27319.72033641],
        [  24902.72584589],
        [  40395.479462  ],
        [  32928.35757595],
        [  10562.01978177],
        [  33826.4980205 ],
        [ 103571.70334649]])

In [9]:
def null_filter(df_sub,block):
    if index == cf.block - 1:
        X_sub = X[:, index * size:]
    else:
        X_sub = X[:, index * size:(index + 1) * size]
    
    for i in xrange(df_sub.shape[1]):
        sum = np.sum(df_sub[:,i])
        print "block {},column {},sum {}".format(str(block),str(i),str(sum))   
    return sum
    
def pickle_data(result):
    print 'save'

In [10]:
X_mat = np.mat(X)
pool =Pool()
s = X_mat.shape[1]/4

for i in xrange(4):
    if i==3:
        r = pool.apply_async(null_filter,(X_mat[:,s*i:],i),callback=pickle_data)
    else:
        r = pool.apply_async(null_filter,(X_mat[:,s*i:s*(i+1)],i),callback=pickle_data)
pool.close()
pool.join()
if r.successful():
    print 'successful'

block 0,column 0,sum 4.92506244047e+14
block 0,column 1,sum 5037522.0
block 0,column 3,sum 5682845.0
block 0,column 2,sum 5735926.0
block 1,column 0,sum 0.0
block 1,column 1,sum 2091115.0
block 1,column 2,sum 11314.0
block 1,column 3,sum 3463671.0
block 2,column 0,sum 105474.73875
block 2,column 1,sum 109515.228365
block 2,column 2,sum 86646.2877743
block 2,column 3,sum 0.0
block 3,column 0,sum 168.332021699
block 3,column 1,sum 86438.5782566
block 3,column 2,sum 6.0
block 3,column 3,sum 7200149.0
save
save
save
save
successful


In [4]:
import os
path=os.path.abspath('../tmp') + '/pickledata'
os.listdir(path)

['0.0m.pkl',
 '2.0m.pkl',
 '2.0+.pkl',
 '1.0-.pkl',
 '1.0d.pkl',
 '3.0l.pkl',
 '2.0*.pkl',
 '0.0log.pkl',
 '2.0log.pkl',
 '2.0x^2+y^2.pkl',
 '2.0l.pkl',
 '1.0x^2+y^2.pkl',
 '3.0d.pkl',
 '1.0m.pkl',
 '3.0x^2+y^2.pkl',
 '2.0-.pkl',
 '3.0log.pkl',
 '0.0d.pkl',
 '3.0+.pkl',
 '2.0d.pkl',
 '3.0-.pkl',
 '1.0+.pkl',
 '0.0l.pkl',
 '3.0*.pkl']

In [28]:
import pickle
import os
import numpy as np

path = os.path.abspath('../tmp') + '/pickledata'
Names = os.listdir(path)
fileNames = [i for i in Names]
file_num = len(fileNames)
print file_num,fileNames

start = 0
feat_sub = 0
#分批从磁盘中加载特征到内存,采用模型或者IV值对交换特征进行排序,并留下topN个特征与下一次加载的特征进行对比
# while(True):
#     if file_num>30:
#         for i in xrange(start*30,(start+1)*30):
#             f = file(path+'/'+fileNames[i],'rb')
#             try:
#                 feat_sub = np.hstack((feat_sub,pickle.load(f)))
#             except:
#                 feat_sub = pickle.load(f)
#             f.close()
#     else:
#         for i in xrange(start*30, file_num):
#             print fileNames[i]
#             f = file(path + '/' + fileNames[i], 'rb')
#             try:
#                 feat_sub = np.hstack((feat_sub, pickle.load(f)))
#             except:
#                 feat_sub = pickle.load(f)
#             f.close()
            
#     start = start+1
#     file_num = file_num-30
#     if file_num <=0:
#         break
# feat_sub.shape
for f in fileNames:
    print f
    a = file(path+'/'+f,'rb')
    arr = pickle.load(a)
    try:
        feat_sub = np.hstack((feat_sub,arr))
    except:
        feat_sub = arr
    a.close()
feat_sub.shape

24 ['0.0m.pkl', '2.0m.pkl', '2.0+.pkl', '1.0-.pkl', '1.0d.pkl', '3.0l.pkl', '2.0*.pkl', '0.0log.pkl', '2.0log.pkl', '2.0x^2+y^2.pkl', '2.0l.pkl', '1.0x^2+y^2.pkl', '3.0d.pkl', '1.0m.pkl', '3.0x^2+y^2.pkl', '2.0-.pkl', '3.0log.pkl', '0.0d.pkl', '3.0+.pkl', '2.0d.pkl', '3.0-.pkl', '1.0+.pkl', '0.0l.pkl', '3.0*.pkl']
0.0m.pkl
2.0m.pkl
2.0+.pkl
1.0-.pkl
1.0d.pkl
3.0l.pkl
2.0*.pkl
0.0log.pkl
2.0log.pkl
2.0x^2+y^2.pkl
2.0l.pkl
1.0x^2+y^2.pkl
3.0d.pkl
1.0m.pkl
3.0x^2+y^2.pkl
2.0-.pkl
3.0log.pkl
0.0d.pkl
3.0+.pkl
2.0d.pkl
3.0-.pkl
1.0+.pkl
0.0l.pkl
3.0*.pkl


(145067, 203)

In [18]:
pickle.load(file(path+'/'+'0.0m.pkl','rb')).shape

(145067, 45)

In [40]:
np.mat(["LDA{}".format(i) for i in range(0, 10, 1)]).shape

(1, 10)